In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys
sys.path.insert(0,'/home/jmamede/scripts/notebooks/libraries/')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [2]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end
#For gui make a function with all this input for verification purposes when loading the files
dirname = "/run/media/jmamede/Joao/CAruby/*.nd2"
# dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"

filelist = glob.glob(dirname)
filelist.sort()
# filelist = filelist[2:3]

print(filelist)

#Load the first file for reference and what's inside the file
frames = ND2Reader(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = 1
ydivide = 1
depthdivide = depth_divide(xdivide, ydivide)


# print(frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep'])


['/run/media/jmamede/Joao/CAruby/F2_laminigfpcaruby_0h.nd2', '/run/media/jmamede/Joao/CAruby/F2_laminigfpcaruby_21h20m.nd2']
{'x': 1024, 'y': 1024, 'c': 3, 't': 1, 'z': 23, 'v': 20}
['FITC (Em)', 'mRuby3', 'mirfp670']


In [3]:
#Need to make this a big more automatic, but indexing all the files and iterations is not easy
#In the version where the PSF is estimated by flowdec is easier because everything was set to automatic

PSFdir="/home/jmamede/Rush/PSF/"
# psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
# psfgfp =  tf.imread(PSFdir+'/green.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647
remove = 8
# remove = 15
psf647 = np.flip(psf647,
#                  axis=0
                )[remove:-remove]
# [8:-8]


psfgfp = np.flip(psfgfp,
#                  axis=0
                )[remove:-remove]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[remove:-remove]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[remove:-remove]

print(psf647.shape)
print(psfruby.shape)
# psf647

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


(19, 128, 128)
(19, 128, 128)


In [4]:
algo =init_RL_algo(psfgfp.ndim
                   ,pad_mode='2357',pad_min=(2,24,24)
#                    ,pad_mode='log2',pad_min=(1,12,12)
                  )


filelist

['/run/media/jmamede/Joao/CAruby/F2_laminigfpcaruby_0h.nd2',
 '/run/media/jmamede/Joao/CAruby/F2_laminigfpcaruby_21h20m.nd2']

In [5]:
n_iters = 100
savePRJ = True
saveStack = True
# time_offset = 0
# maxT = 25
#do not worry about these ones
maxT = None
concattime = False

for fname in filelist:
    #to add the real time of experiment when we have to split it into different files
    if concattime:
#         time_offset = get_deltaT(filelist[0],fname)
        time_offset = get_deltaT(fname1,fname)
#         time_offset = 72424
#         time_offset = 72424
    else: time_offset = 0
    frames =  ND2Reader(fname)
    frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels = frames.metadata['channels']
    metadata = frames.metadata
    
    if saveStack:
        xml = Nd2meta2OMEXML(frames,time_offset=time_offset,maxT=maxT)
    if savePRJ:
        prjxml = Nd2meta2OMEXML(frames, project=True,time_offset=time_offset,maxT=maxT,verbose=False)

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)

    # frames.iter_axes = 'v'
    # for frame in frames:
    for visit in range(22,27):
#         frames.sizes['v']):
#     for visit in range(38,frames.sizes['v']):
#     for visit in [0]:

        frames.default_coords['v'] = visit
        stackfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'_PRJ.ome.tiff'
        if saveStack:
            tifstack = tf.TiffWriter(stackfile, bigtiff=False,imagej=False)
            firststk = True
        if savePRJ:
            tifprj = tf.TiffWriter(prjfile, bigtiff=False, imagej=False)
            first = True
        if maxT == None:
            counterT = frames.sizes['t']
        else: counterT = maxT
        for time in range(counterT):
#         for time in range(25):

#             res = np.zeros(
#                 (frames.sizes['z'],frames.sizes['c']
#                  ,frames.sizes['y'],frames.sizes['x'])
#                 , dtype=np.float32)
    
            res = np.zeros(
                (frames.sizes['z'],frames.sizes['c']
                 ,frames.sizes['y'],frames.sizes['x'])
                , dtype=np.float32)

            for i in range(frames.sizes['c']):
#             for i in range(3, frames.sizes['c']):
                frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                ch = metadata['channels'][i]
                # ['a647', 'DAPI', 'GFP']
                # ['FITC', 'mRuby3', 'miRFP670']
                # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                # ['a647', 'DAPI', 'GFP']
                if ch == 'DAPI' or ch  == '470 nm':
                    psf = psfdapi
                elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM' or ch == '640 nm':
                    psf = psf647
                elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM' or ch == 'FITC (Em)':
                    psf = psfgfp
                elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
                    psf = psfruby
                elif ch == 'FRET-gYFP-dsRED':
                    psf = psfruby
                else: psf = psfruby

                arr = da.from_array(frames[time]           
                                    , chunks=chunk_size)

                res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                    dtype='float32', algo=algo,
                                    psf=psf, iters=n_iters, use_ram=False
                                             ).compute(num_workers=1)

                # If we need to swap Channel and Z in the future
                #             img5d = np.swapaxes(img5d,0,1)

                update_progress( (visit+1) * (time+1) / (frames.sizes['v']* counterT) )
                print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                        visit+1, frames.sizes['v']
                        ,time+1,  counterT
                        , i+1, frames.sizes['c']))
# Make Function
            if saveStack:

                if firststk:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
                        description = xml.to_xml(),
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
                    firststk = False
                    print('blup')
                else:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
#                         description = xml.to_xml(),
#                         datetime= True,
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
            if savePRJ:
                if first:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:                  
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )
        if saveStack:
            tifstack.close()
        if savePRJ:
            tifprj.close()

    update_progress(1)

tvzc 0 0 0 0
4.626818259984255 4.626818259984255
z: 3270.85
tvzc 0 0 0 1
4.626818259984255 4.626818259984255
z: 3270.85
tvzc 0 0 0 2
4.626818259984255 4.626818259984255
z: 3270.85
tvzc 0 0 1 0
7.646459359943867 7.646459359943867
z: 3271.35
tvzc 0 0 1 1
7.646459359943867 7.646459359943867
z: 3271.35
tvzc 0 0 1 2
7.646459359943867 7.646459359943867
z: 3271.35
tvzc 0 0 2 0
10.759615270018578 10.759615270018578
z: 3271.85
tvzc 0 0 2 1
10.759615270018578 10.759615270018578
z: 3271.85
tvzc 0 0 2 2
10.759615270018578 10.759615270018578
z: 3271.85
tvzc 0 0 3 0
13.817082670003176 13.817082670003176
z: 3272.35
tvzc 0 0 3 1
13.817082670003176 13.817082670003176
z: 3272.35
tvzc 0 0 3 2
13.817082670003176 13.817082670003176
z: 3272.35
tvzc 0 0 4 0
16.906666759967806 16.906666759967806
z: 3272.85
tvzc 0 0 4 1
16.906666759967806 16.906666759967806
z: 3272.85
tvzc 0 0 4 2
16.906666759967806 16.906666759967806
z: 3272.85
tvzc 0 0 5 0
19.998896069973707 19.998896069973707
z: 3273.35
tvzc 0 0 5 1
19.9988

KeyError: 506

In [6]:
# print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

<weakproxy at 0x7fe213c4ddd0 to Device at 0x7fe213e23b90>